In [1]:
pwd

'/home/sasha/effective-inference'

In [15]:
import pandas as pd
import timeit
import json

In [4]:
import ast
import re

def extract_function_names_ast(code):
    tree = ast.parse(code)
    function_names = []

    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            function_names.append(node.name)

    return function_names


def extract_function_code_with_regex(code):
    pattern = re.compile(r'\bdef\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\((.*?)\)\s*:\s*(.*?)(?=def|$)', re.DOTALL)
    matches = pattern.findall(code)
    functions = {}

    for match in matches:
        function_name = match[0]
        function_args = match[1]
        function_body = match[2]
        functions[function_name] = f"def {function_name}({function_args}):\n    {function_body}"

    return functions




In [26]:
dataset = pd.read_csv('code_data.csv', index_col=0).reset_index()
dataset = dataset.drop(['level_0'], axis=1)
dataset.head(5)

,index,code,prompt,bad_prompt,bad_code,prompt_names_dict,numerical_prompt,numerical_code,prompt_numerical_dict
0,0,"def cellname(x, y):\n ""Translate a cell coo...","def str(val):\n """"""Convert float to string,...","def locale_aware_float_to_string(val):\n """"...","def generate_cell_name(x, y):\n ""Translate ...","{""str"": ""locale_aware_float_to_string"", ""colnu...","def 0(val):\n """"""Convert float to string, t...","def 2(x, y):\n ""Translate a cell coordinate...","{""str"": ""0"", ""colnum2name"": ""1""}"
1,1,"def test_basic():\n ""Basic non-gui self-tes...","def cellname(x, y):\n ""Translate a cell coo...","def generate_cell_name(x, y):\n ""Translate ...","def perform_basic_non_gui_self_test():\n ""B...","{""cellname"": ""generate_cell_name""}","def 0(x, y):\n ""Translate a cell coordinate...","def 1():\n ""Basic non-gui self-test.""\n ...","{""cellname"": ""0""}"
2,2,"def display_menu(stdscr, menu_y):\n ""Displa...","def erase_menu(stdscr, menu_y):\n ""Clear th...","def erase_menu_clears_menu_space(stdscr, menu_...",def display_menu_commands_and_instructions(std...,"{""erase_menu"": ""erase_menu_clears_menu_space""}","def 0(stdscr, menu_y):\n ""Clear the space w...","def 1(stdscr, menu_y):\n ""Display the menu ...","{""erase_menu"": ""0""}"
3,3,"def merge_profile():\n """"""Reads sys.getdxp(...","def has_pairs(profile):\n """"""Returns True i...","def check_for_pairs(profile):\n """"""Returns ...",def merge_profile_into_module_cached_copy():\n...,"{""has_pairs"": ""check_for_pairs""}","def 0(profile):\n """"""Returns True if the Py...","def 1():\n """"""Reads sys.getdxp() and merges...","{""has_pairs"": ""0""}"
4,4,"def snapshot_profile():\n """"""Returns the cu...","def merge_profile():\n """"""Reads sys.getdxp(...",def merge_profile_into_module_cached_copy():\n...,"def get_cumulative_execution_profile():\n ""...","{""merge_profile"": ""merge_profile_into_module_c...","def 0():\n """"""Reads sys.getdxp() and merges...","def 1():\n """"""Returns the cumulative execut...","{""merge_profile"": ""0""}"


## Make the new func names dataset

In [12]:
func_data = pd.DataFrame({'func_name': [],'func_definition': [], 'prompt': [], 'code': [] })
for i in range(dataset.shape[0]):
  line = dataset.loc[i]
  rows = []
  for func_name, func in extract_function_code_with_regex(line['prompt']).items():
    if func_name in line['code']:
      rows.append({'func_name': func_name,'func_definition':func,  'prompt': line['prompt'], 'code': line['code'], 'count': line['code'].count(func_name)})

  func_data = pd.concat([func_data, pd.DataFrame(rows)], ignore_index=True)

lst = '''parsedate_tz -> _parsedate_tz
make_translate -> make_translate
neg -> neg
grid_values -> grid_values
assign -> assign
eliminate -> eliminate
shuffled -> shuffled
assign -> assign
random_puzzle -> random_puzzle
get_dot_atom_text -> parse_dot_atom_text
get_domain -> parse_domain
get_display_name -> parse_display_name
get_extended_attrtext -> parse_extended_attrtext
get_extended_attribute -> parse_extended_attribute
Value -> synchronized_value
parse_mime_parameters -> parse_mime_parameters_from_value
_qencode -> quopri_encode
_format_timetuple_and_zone -> format_timetuple_and_zone
_parsedate_tz -> parse_date_with_timezone
vals_sorted_by_key -> values_sorted_by_key
_unpack_uint32 -> unpack_uint32
_install_handlers -> install_handlers_from_config
_resolve -> resolve_dotted_name
_strip_spaces -> strip_spaces_from_list
release -> get_system_release
_get_ptext_to_endchars -> get_ptext_until_endchars
Comment -> create_comment_element
get_cfws -> parse_cfws_from_value
_create_carefully -> create_carefully_and_open
_format -> format_with_options
h -> quadratic_function
prepend_module_to_path -> prepend_module_to_sys_path
from_qvariant -> convert_qvariant_to_python
get_prog -> get_program_name
marker -> parse_marker
get_unpatched -> retrieve_unpatched_item
liberal_is_HDN -> is_host_domain_name
escape_path -> escape_invalid_path_characters
addsitedir -> add_site_directory
getsitepackages -> get_site_packages_directories
addsitedir -> add_site_directory
abspath -> get_absolute_path
_joinrealpath -> join_real_path
_read_output -> read_command_output
_get_system_version_tuple -> get_system_version_tuple
S_IFMT -> get_file_mode_type
decode_long -> decode_twos_complement_long
makepath -> normalize_and_absolute_path
_init_pathinfo -> initialize_path_info
addpackage -> process_site_packages_pth
_get_path -> get_site_packages_path
getuserbase -> get_user_base
getusersitepackages -> get_user_site_packages
p -> parse_default_parameter
getfileinfo -> get_file_info
read_stringnl_noescape -> read_string_no_escape_nl
read_uint1 -> read_uint_1_byte
read_int4 -> read_int_4_bytes
read_uint4 -> read_uint_4_bytes
read_stringnl -> read_string_nl
LParen -> create_left_paren_leaf
find_binding -> find_variable_binding_node
find_root -> find_top_level_namespace_node
_generate_pickle_name -> generate_pickle_filename
_newer -> is_file_a_newer_than_b
load_grammar -> load_grammar_tables
get_lineno -> get_line_number_in_text_widget
idle_formatwarning -> format_warning_idly
showerror -> show_error_message
get_spaces_firstword -> extract_whitespace_and_first_word
reformat_paragraph -> reformat_text_paragraph
_pack_uint32 -> convert_uint32_to_little_endian_bytes
_get_supported_file_loaders -> get_supported_file_loaders_list
open_binary -> open_binary_resource
_path_from_reader -> get_path_from_reader
getparser -> create_parser_and_unmarshaller
_get_head_types -> get_head_types_from_pattern
get_all_fix_names -> get_all_fix_names_in_package
Dot -> create_dot_leaf
ArgList -> create_argument_list_node
Attr -> create_attribute_node
Name -> create_name_leaf
RParen -> create_right_paren_leaf
enabled -> use_distutils_local
_load_sysconfig_schemes -> load_sysconfig_schemes
_pypy_hack -> pypy_name_for_platform
warn_distutils_present -> warn_distutils_module_present_but_use_setuptools
enabled -> distutils_selection_by_env_var
_show -> message_box_show_with_icon_type_and_options
_load_scheme -> load_scheme_by_name
cache -> simple_unbounded_cache
_zip_equal -> zip_equal_or_raise_error
_chunked_even_finite -> chunk_even_finite_iterable
str -> convert_float_to_string
colnum2name -> convert_column_number_to_name
cellname -> generate_cell_name
has_pairs -> check_for_pairs
enumerate -> list_all_alive_thread_objects
has_pairs -> check_for_pairs
enumerate -> list_all_alive_thread_objects
common_pairs -> find_most_common_opcode_pairs
snapshot_profile -> get_cumulative_execution_profile
html_highlight -> convert_classified_text_to_html
alltt_escape -> replace_backslash_and_braces_with_escaped_equivalents
gettempdirb -> get_tempdir_as_bytes
_infer_return_type -> infer_implied_return_type
gettempdir -> get_tempdir_as_string
_get_candidate_names -> get_candidate_names
mksalt -> generate_salt
iter_importers -> yield_finders_for_module
get_importer -> retrieve_finder_for_path_item
str -> convert_float_to_string
find_lines -> search_for_lines
_find_lines_from_code -> search_for_lines_from_code
_try_compile -> attempt_compile
_format_code_info -> generate_code_info_format
_get_code_object -> extract_code_object
_try_compile -> attempt_compile
_get_code_object -> extract_code_object
_get_code_object -> obtain_code_object
findlinestarts -> locate_line_starts
_try_compile -> attempt_compile
b64encode -> encode_base64
b64decode -> decode_base64
_bytes_from_decode_data -> extract_bytes_from_decode_data
_85encode -> encode_85
str -> convert_float_to_string
_readmodule -> process_module_read
lru_cache -> fancy_cache_decorator
__get_builtin_constructor -> fetch_builtin_constructor
_maybe_compile -> potentially_compile
fnmatchcase -> name_match_case
match -> try_match
print_list -> display_extracted_list
extract_tb -> harvest_traceback_entries
_parse_value_tb -> _dissect_error_traceback
format_exception -> mold_exception_as_text
_formatwarnmsg_impl -> _craft_warning_message
_showwarnmsg_impl -> _present_warning_message
_is_internal_frame -> _looks_like_internal_frame
call -> shout_command
getstatusoutput -> retrieve_command_status_and_output
getincrementalencoder -> fetch_incremental_encoder
getincrementaldecoder -> fetch_incremental_decoder
enumerate -> get_all_alive_threads
makedirs -> create_directory_recursive
_execvpe -> _execute_via_path_variable
_isclass -> _is_non_generic_class
getpager -> decide_paging_method
_escape_stdout -> _encode_stdout_with_escape
plain -> remove_bold_formatting
_get_revised_path -> _adjust_path_for_current_directory
pdatecache -> update_source_cache
prepare_class -> prepare_metaclass_and_namespace
resolve_bases -> resolve_mro_entries_dynamically
enumerate -> get_all_alive_threads
calculate_meta -> calculate_most_derived_metaclass
wraps -> apply_update_wrapper_to_wrapper
parse227 -> parse_pasv_response
current_thread -> get_current_thread
active_count -> get_number_of_active_threads
mac_ver_xml -> get_mac_version_xml_info
filter -> filter_names_by_pattern
follow_symlinks -> follow_symlinks_until_real_file
uname -> get_platform_information
uname -> get_platform_information
_sys_version -> parse_python_sys_version
add_callers -> combine_caller_lists
isleap -> is_leap_year
weekday -> get_weekday
_has_code_flag -> has_code_flag
isbuiltin -> is_builtin_function_or_method
ismethoddescriptor -> is_method_descriptor
istraceback -> is_traceback
isframe -> is_frame
ismodule -> is_module
getsourcefile -> get_source_file
walktree -> walk_tree
getargs -> get_arguments
getframeinfo -> get_frame_info
getouterframes -> get_outer_frames
getinnerframes -> get_inner_frames
isbuiltin -> is_builtin_function_or_method
_signature_is_builtin -> is_builtin_signature
_astuple_inner -> convert_object_to_tuple_recursively
_deduplicate -> remove_duplicates_preserve_order
_eval_type -> evaluate_type_with_forward_references
_strip_annotations -> remove_type_annotations_recursively
str -> convert_float_to_locale_aware_string
_is_leap -> check_if_year_is_leap
_is_leap -> check_if_year_is_leap
_days_before_year -> calculate_days_before_start_of_year
_days_before_month -> calculate_days_before_start_of_month
_days_in_month -> get_number_of_days_in_month
getcontext -> get_current_thread_context_or_create_new
str -> convert_float_to_locale_aware_string
abs -> calculate_absolute_value
str -> convert_float_to_locale_aware_string
wraps -> create_decorator_with_update_wrapper
_isfinite -> check_if_value_is_finite
T -> calculate_punycode_parameter
bisect_left -> find_index_to_insert_left_sorted
bisect_right -> find_index_to_insert_right_sorted
_sum -> calculate_high_precision_sum
_convert -> convert_value_to_numeric_type
enumerate -> get_list_of_currently_alive_threads
fmean -> calculate_arithmetic_mean_of_floats
variance -> calculate_sample_variance
pvariance -> calculate_population_variance
_asdict_inner -> convert_object_to_dict_recursively
_is_dataclass_instance -> check_if_object_is_dataclass_instance
_localize -> format_number_with_locale
delocalize -> parse_number_string_with_locale
_parse_localename -> parse_locale_code
_build_localename -> build_locale_code
__import__ -> import_module
_normalize_module -> normalize_module
script_from_examples -> generate_script_from_examples
fallback_getpass -> get_password_with_warning
_getattribute -> get_attribute_recursive
_open_terminal -> open_pty_master_and_get_terminal
slave_open -> open_pty_slave_and_acquire_terminal
wait -> wait_for_futures
repeat -> timeit_repeat
_parse_mac -> parse_mac_address
_is_universal -> is_universal_mac
_find_mac_near_keyword -> find_mac_address_near_keyword
_find_mac_under_heading -> find_mac_address_under_heading
_windll_getnode -> get_hardware_address_windows
_builtin_from_name -> load_builtin_module
_readmailcapfile -> read_mailcap_file
parsefield -> parse_mailcap_field
_run_code -> run_code_in_namespace
selective_len -> count_chars_below_max
selective_find -> find_next_occurrence
T  -> calculate_punycode_parameter
enumerate -> get_alive_threads
generate_generalized_integer -> generate_generalized_variable_length_integer
adapt -> calculate_punycode_bias
T -> calculate_punycode_parameter
decode_generalized_number -> decode_generalized_variable_length_integer
adapt -> calculate_punycode_bias
str -> format_float_locale_aware
filter -> filter_names
_find_exe_version -> get_executable_version
_get_xxmodule_path -> get_xxmodule_path
get_ld_header -> get_ld_header_from_process
get_ld_header_info -> get_ld_header_info_from_process
filter -> filter_names
get_one_match -> get_first_matching_group
get_legacy -> get_legacy_naming_scheme
get_version -> get_highest_versioned_member
get_ld_headers -> parse_ld_headers
get_member -> get_matching_archive_member
get_shared -> extract_shared_objects
_other_endian -> get_opposite_endian_type
dyld_image_suffix -> get_dyld_image_suffix
dyld_image_suffix_search -> add_dyld_image_suffix_semantics
get_logger -> get_multiprocessing_logger
_run_finalizers -> run_finalizers_with_minpriority
address_type -> get_address_type
_fixup_main_from_path -> fixup_main_module_from_path
_new_value -> _create_new_value
_new_value -> _create_new_value
RawValue -> create_shared_memory_object
synchronized -> create_synchronized_object
convert_to_error -> handle_conversion_to_error
all_methods -> get_all_method_names
MakeProxyType -> create_proxy_type
dispatch -> send_message_and_receive_response
urlparse -> parse_url
_splitport -> split_host_and_port
getproxies_environment -> get_environment_proxies
_coerce_args -> _coerce_and_create_args
urlunsplit -> unsplit_url
_coerce_args -> _coerce_and_create_args
requires_to_requires_dist -> get_version_specifier_for_requirement
pkginfo_unicode -> coax_unicode_from_pkginfo
_should_cache -> can_cache_built_requirement
input -> create_file_input_instance
call_subprocess -> execute_subprocess
_test_writable_dir_win -> is_writable_directory_on_windows
get_best_invocation_for_this_python -> get_current_python_invocation
glibc_version_string_confstr -> get_glibc_version_string_using_confstr
glibc_version_string_ctypes -> get_glibc_version_string_using_ctypes
split_leading_dir -> split_first_directory_from_path
et_prog -> retrieve_program_and_handle_special_cases
create_package_set_from_installed -> generate_set_of_installed_packages_with_dependency_details
_create_whitelist -> derive_whitelist_based_on_installed_and_provided_packages
_simulate_installation_of -> predict_installed_packages_after_simulation
check_package_set -> validate_consistency_of_package_set
_check_download_dir -> verify_download_directory_for_existing_file_with_correct_hash
make_setuptools_develop_args -> generate_setuptools_develop_arguments_with_options
call_subprocess -> execute_subprocess_and_handle_return_code
format_command_result -> prepare_command_result_for_logging
runner_with_spinner_message -> execute_runner_with_spinner_displaying_message
error -> log_error_message
_module_repr_from_spec -> get_module_repr_from_spec
spec_from_file_location -> create_module_spec_from_file_location
_init_module_attrs -> initialize_module_attributes
_new_module -> create_new_module_instance
_load_unlocked -> load_module_unlocked
_find_and_load_unlocked -> find_and_load_module_unlocked
_resolve_name -> resolve_relative_module_name
_find_and_load -> find_and_load_module
_path_stat -> stat_path
_path_is_mode_type -> check_path_mode_type
check_first_requirement_in_file -> validate_first_line_as_requirements_file
is_installable_dir -> identify_directory_as_installable_project
_clean_url_path -> sanitize_url_path
wrapper -> initialize_curses_and_call_function
_create_link_from_element -> convert_anchor_element_attributes_to_link
_determine_base_url -> identify_base_url_from_html_document
_find_name_version_sep -> locate_separator_index_based_on_canonical_name
get_close_matches -> retrieve_similar_matches_using_sequence_matcher
ascii_lower -> transform_ascii_to_lower_with_unicode_support
_iter_decode_generator -> generate_decoded_output_iterator_using_decoder
get_filetype_from_line -> determine_filetype_from_last_line
doctype_matches -> check_if_doctype_matches_regex
guess_decode -> attempt_decoding_with_guess
regex_opt_inner -> generate_inner_regex_for_string_list
_apply -> apply_configuration_to_distribution
find_filter_class -> search_for_filter_class_by_name
guess_decode -> attempt_decoding_with_guess
get_lexer_by_name -> retrieve_lexer_by_alias
get_filetype_from_buffer -> determine_filetype_from_buffer
find_formatter_class -> look_up_formatter_class_by_alias_returning_none_if_not_found
load_system -> load_build_system_from_source_directory_pyproject_toml
check_call -> run_command_wait_for_completion_raise_exception_if_nonzero_exit_code
_build_backend -> find_and_load_build_backend_add_in_tree_backend_directories
_find_already_built_wheel -> check_for_already_built_wheel_get_wheel_metadata_hook
_build_backend -> find_and_load_build_backend_add_in_tree_backend_directories
_find_already_built_wheel -> check_for_already_built_wheel_get_wheel_metadata_hook
_build_backend -> find_and_load_build_backend_add_in_tree_backend_directories
parse_marker -> parse_marker_string_returning_dictionary_with_expression
finder -> return_resource_finder_for_package_with_caching
cookiejar_from_dict -> create_cookie_jar_from_key_value_dict_with_overwrite_option
to_key_val_list -> convert_object_to_list_of_tuples_if_possible
merge_setting -> determine_appropriate_setting_for_request_and_session_with_dictionary_merging
unquote_header_value -> reverse_quote_header_value_based_on_browser_implementation
_parse_content_type_header -> parse_content_type_header_returning_content_type_and_parameters
get_encoding_from_headers -> extract_encodings_from_http_header_dictionary
dotted_netmask -> convert_mask_from_slash_xx_format_to_quad_dotted_format
get_environ_proxies -> return_environment_proxies_dictionary_with_bypass_logic
parse_url -> parse_url_into_namedtuple_with_rfc3986_compliance
urlunparse -> put_together_parsed_url_with_redundant_delimiters_handling
is_dataclass -> check_if_object_is_dataclass_or_instance
cleandoc -> clean_up_indentation_from_docstrings
cell_len -> get_number_of_cells_required_to_display_text
get_host -> deprecated_use_parse_url_instead_get_scheme_host_and_port
resolve_cert_reqs -> resolve_argument_to_numeric_constant_for_cert_reqs
resolve_ssl_version -> resolve_argument_to_numeric_constant_for_ssl_version
wait_for_socket -> choose_wait_for_socket_implementation_delayed
_encode_invalid_chars -> _percent_encode_uri_component_without_reapplying_on_existing_percent_encoding
wait_for_read -> wait_for_socket_to_become_readable
_to_unicode -> convert_object_to_unicode_for_python2
is_appengine -> check_if_running_on_appengine
_cfstr -> create_cfstring_from_python_binary_data
_cf_string_to_unicode -> convert_cfstring_to_unicode_string
interpret_distro_name -> generate_alternative_interpretations_of_source_distro_name
egg_info_for_url -> extract_egg_info_from_url
distros_for_location -> yield_egg_or_source_distributions_based_on_basename
_resolve_tar_file_or_dir -> resolve_links_and_extract_link_targets_as_normal_files
find_package_path -> resolve_package_path_with_package_dir
_get_mro -> get_sorted_mro_of_bases
isdir -> check_if_path_is_existing_directory
_msvc14_get_vc_env -> get_vc_environment_variables_for_msvc14
_get_option -> get_target_option
_attrgetter -> safe_attr_getter
glob_relative -> expand_globs_relative
is_python -> check_valid_python_script
_safe -> escape_wheel_component
str -> locale_aware_float_to_string
_list_from_layouttuple -> construct_list_from_ttk_layout_tuple
_splitdict -> format_dict_from_tcl_list_pairs
_format_optdict -> format_option_dict_to_tuple_for_tk_call
_convert_stringval -> convert_value_to_more_appropriate_python_object
_list_from_statespec -> construct_list_from_state_spec_tuple
_tclobj_to_py -> convert_tcl_object_to_python_object
__dict_replace -> replace_substrings_using_dictionary
_do_pulldom_parse -> perform_pulldom_parse_and_get_root_node
_read_headers -> read_headers_into_list_from_file_pointer
_str2time -> convert_string_to_time
isCocoaTk -> is_using_cocoa_tk
readSystemPreferences -> read_macos_system_preferences
isAquaTk -> is_using_native_osx_tk
_ensure_future -> ensure_future_internal
ensure_future -> ensure_future_wrap
_get_running_loop -> get_running_event_loop
get_event_loop_policy -> get_current_event_loop_policy
_convert_future_exc -> convert_future_exception
isfuture -> is_future_instance
_create_formatters -> create_log_formatters
_find_lines -> get_line_numbers_for_code_objects
samestat -> are_stat_buffers_equal
getlines -> get_lines_from_cache
updatecache -> update_cache_entry
_calculate_meta -> calculate_most_derived_metaclass
_mac_ver_xml -> get_mac_version_info_from_xml
_follow_symlinks -> resolve_symlinks
_signature_fromstr -> parse_signature_from_string
erase_menu -> erase_menu_clears_menu_space
merge_profile -> merge_profile_into_module_cached_copy
urlopen -> urlopen_open_url_with_data_and_timeout
_get_default_tempdir -> _calculate_default_temporary_directory
_gettempdir -> _private_accessor_for_tempfile_tempdir
disassemble -> disassemble_code_object
_get_instructions_bytes -> _iterate_over_instructions_in_bytecode_string
_disassemble_bytes -> _disassemble_code_bytes
_disassemble_recursive -> _recursive_disassembly_of_code_object
_unpack_opargs -> _unpack_opcode_arguments
_write_file_prefix -> _write_shebang_line
_input_type_check -> _check_input_type
_aix_tag -> _infer_abi_bitwidth_for_aix
_aix_bgt -> _get_aix_build_gnu_type
_siftdown -> _sift_down_heap
_siftup -> _sift_up_heap
_siftup_max -> _sift_up_max_heap
_siftdown_max -> _sift_down_max_heap
function -> function_with_default_argument_Foo
_compose_mro -> _calculate_method_resolution_order
walk_tb -> walk_traceback_frames_and_line_numbers
walk_stack -> walk_stack_frames_and_line_numbers
removedirs -> remove_directory_and_empty_intermediate_dirs
execvp -> execute_executable_with_argument_list
execvpe -> execute_executable_with_argument_list_and_environment
pager -> determine_and_use_appropriate_pager
writedoc -> write_html_documentation_to_file
clearcache -> clear_entire_cache
_strptime -> _parse_time_from_string_with_format
__methodDict -> helper_function_for_scrolled_canvas
isclass -> check_if_object_is_class
isfunction -> determine_if_object_is_user_function
ismethod -> verify_if_object_is_instance_method
iscode -> check_if_object_is_code_object
getfile -> identify_source_or_compiled_file_of_object
isgenerator -> validate_if_object_is_generator
_genops -> generate_operations_from_data
_trace -> log_trace_message_if_verbose
token_bytes -> generate_random_byte_string_of_length
_find_executable -> search_for_executable_in_path_directories
_get_system_version -> retrieve_os_x_system_version_as_string
_save_modified_value -> persist_modified_and_original_values_of_configuration_variable
_sync_flush -> ensure_changes_to_file_are_physically_on_disk
getdefaultlocale -> attempt_to_determine_default_locale
debug_script -> debug_test_script
close -> close_the_sequence
_writen -> write_all_data_to_descriptor
no_type_check -> indicate_that_annotations_are_not_type_hints
_get_command_stdout -> obtain_stdout_of_command_with_args
_validate_tzfile_path -> validate_timezone_file_path
dyld_default_search -> search_for_library_using_dyld_default_semantics
dyld_executable_path_search -> search_for_library_using_executable_path
dyld_override_search -> override_search_for_library_using_dyld_semantics
dyld_find -> find_library_using_dyld_semantics
skip -> unconditionally_skip_test_due_to_reason
seal -> disable_automatic_generation_of_child_mocks
install_translator -> install_qt_translator_to_qapplication
convert_requirements -> generate_requires_dist_strings_for_parsed_requirements
_check_no_input -> check_for_no_input_and_raise_error
join_lines -> join_lines_ending_with_backslash_except_after_comments
expand_env_variables -> expand_environment_variables_in_lines_enum
raise_for_status -> check_and_raise_for_status
_ensure_html_header -> validate_and_ensure_html_header
_iter_encode_generator -> iterate_and_encode_generator
find_plugin_filters -> discover_and_return_plugin_filters
find_plugin_lexers -> search_and_retrieve_plugin_lexers
_load_lexers -> load_and_cache_lexers
find_lexer_class_for_filename -> search_and_get_lexer_class_for_filename
_iter_lexerclasses -> iterate_over_lexer_classes
_load_formatters -> load_and_cache_formatters
find_plugin_formatters -> discover_and_return_plugin_formatters
find_plugin_styles -> discover_and_return_plugin_styles
_get_wheel_metadata_from_wheel -> extract_metadata_from_wheel
make_graph -> generate_dependency_graph
create_cookie -> generate_cookie
request -> construct_and_send_request
should_bypass_proxies -> check_should_bypass_proxies
default_user_agent -> get_default_user_agent
inspect -> inspect_python_object
_replace_multiple -> replace_multiple_occurrences
create_urllib3_context -> generate_urllib3_context
_validate_dependencies_met -> validate_pyopenssl_dependencies
find_library -> search_library_on_aix
yield_lines -> generate_non_empty_lines
_unpack_zipfile_obj -> extract_from_zipfile_object
_iter_open_tar -> iterate_and_open_tar
_clear_modules -> clear_modules_list
_augment_exception -> add_details_to_exception
assert_string_list -> verify_string_list
cleanup -> perform_cleanup_operations
disable_stdlib_finder -> disable_stdlib_finder_by_monkey_patching
validator -> validate_wsgi_compliance
reduce -> cumulative_reduction
sorted_walk -> reproducible_os_walk
iter_symbols -> yield_code_symbols
_chunked_even_online -> yield_chunked_even_online
ensure_local_distutils -> ensure_local_distutils_with_import
_get_default_root -> get_default_root_or_support
is_HDN -> is_host_domain_name
deepvalues -> iterate_over_nested_mapping_sorted
_bootstrap -> bootstrap_pip_into_python
_get_module_lock -> get_or_create_module_lock
_wrap -> simple_substitute_for_update_wrapper
_lock_unlock_module -> acquire_and_release_module_lock
_sanity_check -> verify_sane_arguments
_set_bootstrap_module -> set_bootstrap_module
rec_test -> recursive_test_on_sequence
attr_chain -> follow_attribute_chain
generate_matches -> generate_matches_for_patterns
has_metaclass -> check_class_metaclass
tokenize_loop -> loop_over_tokens
fix_scaling -> scale_fonts_on_hidpi_displays
architecture -> query_executable_architecture
_init_tk_type -> initialize_osx_tk_variant_values
fixb2context -> remove_bad_aquatk_bindings
addOpenEventSupport -> add_open_event_support
overrideRootMenu -> replace_root_menu_for_idle
hideTkConsole -> try_hide_tk_console
close_subprocess_debugger -> close_debugger_subprocess
_set_task_name -> set_task_name_if_not_none
_init_event_loop_policy -> _initialize_event_loop_policy_with_default
_chain_future -> _chain_two_futures_and_copy_result
_format_callbacks -> _format_future_callbacks
_clearExistingHandlers -> _clear_and_close_existing_logging_handlers
_install_loggers -> _create_and_install_logging_loggers
_acquireLock -> _acquire_module_level_lock_for_serializing_access
_releaseLock -> _release_module_level_lock_acquired_by_acquireLock
basicConfig -> do_basic_configuration_for_logging_system
critical -> log_critical_message_on_root_logger
error -> log_error_message_on_root_logger
_validate_xtext -> _validate_xtext_and_register_defect_if_non_printables
get_comment -> parse_and_get_comment_from_value
get_fws -> parse_and_get_folding_whitespace_from_value
get_qp_ctext -> parse_and_get_quoted_printable_ctext_from_value
get_bare_quoted_string -> parse_and_get_bare_quoted_string_from_value
get_encoded_word -> parse_and_get_encoded_word_from_value
get_atext -> parse_and_get_atext_from_value
get_local_part -> parse_and_get_local_part_from_value
get_phrase -> parse_and_get_phrase_from_value
get_angle_addr -> parse_and_get_angle_addr_from_value
get_dtext -> parse_and_get_dtext_from_value
get_msg_id -> parse_and_get_message_id_from_value
get_unstructured -> parse_and_get_unstructured_from_value
get_ttext -> parse_and_get_ttext_from_value
get_attrtext -> parse_and_get_attrtext_from_value
get_quoted_string -> parse_and_get_quoted_string_from_value
_init_event_loop_policy -> initialize_global_event_loop_policy
_chain_future -> chain_two_futures
_format_callbacks -> format_callbacks_helper
_clearExistingHandlers -> clear_and_close_existing_handlers
_install_loggers -> configure_and_install_loggers
_acquireLock -> acquire_module_level_lock
_releaseLock -> release_module_level_lock
basicConfig -> do_basic_logging_configuration
critical -> log_critical_message_root_logger
error -> log_error_message_root_logger
_validate_xtext -> validate_xtext_input_token
get_comment -> parse_comment_from_value
get_fws -> parse_fws_from_value
get_qp_ctext -> parse_qp_ctext_from_value
get_bare_quoted_string -> parse_bare_quoted_string
get_encoded_word -> parse_encoded_word_from_value
get_atext -> parse_atext_from_value
get_local_part -> parse_local_part_from_value
get_phrase -> parse_phrase_from_value
get_angle_addr -> parse_angle_addr_from_value
get_dtext -> parse_dtext_from_value
get_msg_id -> parse_msg_id_from_value
get_unstructured -> parse_unstructured_value
get_ttext -> parse_ttext_from_value
get_attrtext -> parse_attrtext_from_value
get_quoted_string -> parse_quoted_string_from_value
test_basic -> perform_basic_non_gui_self_test
display_menu -> display_menu_commands_and_instructions
common_instructions -> get_most_common_opcodes
render_common_pairs -> render_most_common_opcode_pairs
seq -> generate_sequence_of_opcode_pairs
build_html_page -> create_html_page_with_colorized_code
latex_highlight -> create_latex_document_with_colorized_code
get_json -> download_and_decode_json_from_url
_sanitize_params -> sanitize_and_infer_params_for_api
_mkstemp_inner -> create_temporary_file_and_return_fd
crypt -> return_one_way_hash_with_salt
iter_modules -> iterate_over_modules_on_path
_split_optional_netmask -> split_netmask_and_raise_error_if_needed
distb -> disassemble_traceback
code_info -> format_details_of_methods_or_functions
get_instructions -> iterate_over_opcodes_in_code
_disassemble_str -> compile_and_disassemble_source_string
findlabels -> detect_offsets_of_jump_targets
_copy_archive -> copy_application_archive_with_shebang_modification
standard_b64encode -> encode_bytes_using_standard_base64_alphabet
standard_b64decode -> decode_bytes_with_standard_base64_alphabet
urlsafe_b64encode -> encode_bytes_using_urlsafe_base64
urlsafe_b64decode -> decode_bytes_using_urlsafe_base64
b16decode -> decode_bytes_with_base16
b85encode -> encode_bytes_in_base85_format
encodebytes -> encode_bytestring_into_base64_data
decodebytes -> decode_base64_data_into_bytestring
_aix_bosmp64 -> get_version_and_build_date_of_aix_bosmp64
aix_buildtag -> return_platform_tag_of_system_built_on
readmodule -> return_classes_in_module
execlpe -> execute_and_replace_current_process_environment
isdata -> check_if_object_represents_data
sort_attributes -> sort_attrs_in_place_by_fields_and_alphabetically
plainpager -> simply_print_unformatted_text_ultimate_fallback
describe -> produce_short_description_of_given_thing
writedocs -> write_html_documentation_for_all_modules
_adjust_cli_sys_path -> ensure_current_dir_on_sys_path_and_exclude_main
_keep_alive -> keep_reference_to_object_in_memo
commonprefix -> find_longest_common_leading_component
samefile -> test_whether_pathnames_reference_same_file_or_directory
sameopenfile -> test_whether_open_file_objects_reference_same_file
getline -> get_line_for_python_source_file_from_cache
new_class -> create_class_object_dynamically_using_appropriate_metaclass
raise_conversion_error -> wrap_any_raised_struct_errors_in_conversion_error
result -> handle_result_with_conversion_error
_strptime_time -> return_time_struct_based_on_input_string_and_format
_strptime_datetime -> return_datetime_instance_based_on_input_string_and_format
ftpcp -> copy_file_from_one_ftp_instance_to_another
currentThread -> return_current_thread_object_corresponding_to_callers_thread
activeCount -> return_number_of_thread_objects_currently_alive
__methods -> helper_function_for_scrolled_canvas
mac_ver -> get_macos_version_information_as_tuple
_platform -> helper_to_format_platform_string_in_filename_compatible_format
_syscmd_file -> interface_to_systems_file_command
system -> returns_the_system_or_os_name
node -> returns_the_computers_network_name
machine -> returns_the_machine_type
processor -> returns_the_true_processor_name
python_implementation -> returns_string_identifying_python_implementation
python_version -> returns_python_version_as_string
readmodule_ex -> retrieve_module_functions_and_classes_without_importing
insort_right -> insert_item_to_right_in_sorted_list_maintaining_order
insort_left -> insert_item_to_left_in_sorted_list_maintaining_order
heappush -> add_item_to_heap_maintaining_heap_invariant
heappop -> remove_and_return_smallest_item_from_heap_maintaining_invariant
heapreplace -> pop_and_return_smallest_then_add_new_item_to_heap
heappushpop -> fast_version_of_push_followed_by_pop_from_heap
heapify -> transform_list_into_heap_in_place
_heappop_max -> max_heap_version_of_pop_from_heap
_heapreplace_max -> max_heap_version_of_pop_followed_by_push_to_heap
_heapify_max -> transform_list_into_max_heap_in_place
_find_impl -> find_best_matching_implementation_from_registry_for_type
__py_new -> create_new_hashing_object_with_named_algorithm
__hash_new -> create_new_hashing_object_with_named_algorithm
compile_command -> compile_command_and_determine_if_incomplete
fnmatch -> test_whether_filename_matches_pattern
print_tb -> print_stack_trace_entries_up_to_limit
format_tb -> format_stack_trace_entries_up_to_limit
format_exc -> format_exception_as_string_up_to_limit
extract_stack -> extract_stack_with_limit_replacement_for_asyncio
formatwarning -> format_warning_the_standard_way
_showwarnmsg -> hook_to_write_warning_to_file_replace_if_needed
_formatwarnmsg -> function_to_format_warning_the_standard_way
_next_external_frame -> find_next_frame_not_involving_cpython_internals
getoutput -> return_output_of_executing_command_in_shell
iterencode -> encoding_iterator_using_incremental_encoder
iterdecode -> decoding_iterator_using_incremental_decoder
indexOf -> return_first_index_of_item_in_sequence
renames -> rename_old_to_new_across_devices_handling_directories
execlp -> execute_executable_file_along_path_replacing_current_process
python_version_tuple -> retrieve_python_version_as_tuple_of_strings_with_patchlevel
python_branch -> retrieve_python_implementation_branch_string
python_revision -> retrieve_python_implementation_revision_string
python_build -> retrieve_python_build_number_and_date_as_strings
python_compiler -> retrieve_compiler_used_for_compiling_python
add_func_stats -> combine_stats_for_two_profile_entries_with_callers
monthrange -> return_weekday_and_number_of_days_for_year_and_month
isdatadescriptor -> check_if_object_is_data_descriptor_return_true_or_false
isgeneratorfunction -> return_true_if_object_is_user_defined_generator_function
isasyncgenfunction -> return_true_if_object_is_asynchronous_generator_function
isroutine -> return_true_if_object_is_any_kind_of_function_or_method
getabsfile -> return_absolute_path_to_source_or_compiled_file_for_object
getclasstree -> arrange_list_of_classes_into_nested_hierarchy_of_lists
getargvalues -> get_information_about_arguments_passed_into_frame
stack -> return_list_of_records_for_stack_above_callers_frame
trace -> return_list_of_records_for_stack_below_current_exception
getgeneratorlocals -> get_mapping_of_generator_local_variables_to_values
_signature_is_functionlike -> test_if_object_is_duck_type_of_function_type
_signature_from_builtin -> get_signature_for_builtin_callables
binhex -> create_binhex_encoded_copy_of_file
read_stringnl_noescape_pair -> read_stringnl_noescape_and_return_pair
read_string1 -> read_string1_with_bytes_input
read_string4 -> read_string4_with_bytes_input
read_bytes1 -> read_bytes1_with_bytes_input
read_bytes4 -> read_bytes4_with_bytes_input
read_unicodestringnl -> read_unicodestringnl_with_bytes_input
read_unicodestring1 -> read_unicodestring1_with_bytes_input
read_decimalnl_short -> read_decimalnl_short_with_bytes_input
read_decimalnl_long -> read_decimalnl_long_with_bytes_input
read_floatnl -> read_floatnl_with_bytes_input
read_long1 -> read_long1_and_return_decoded_value_from_bytes_input
read_long4 -> read_long4_and_return_decoded_value_from_bytes_input
genops -> generate_all_opcodes_in_pickle_and_return_opcode_info
removeduppaths -> remove_duplicate_entries_from_sys_path_and_make_absolute
addusersitepackages -> add_per_user_site_package_to_sys_path
addsitepackages -> add_site_packages_to_sys_path
recursive_repr -> decorator_to_make_repr_return_fillvalue_for_recursive_call
decorating_function -> function_decorated_to_handle_recursive_repr
token_hex -> return_random_text_string_in_hexadecimal_with_nbytes
token_urlsafe -> return_random_URL_safe_text_string_in_Base64_encoding_with_nbytes
realpath -> return_canonical_path_of_specified_filename_eliminating_symbolic_links
_find_build_tool -> find_build_tool_on_current_path_or_using_xcrun
_default_sysroot -> return_the_root_of_the_default_sysroot_for_compiler
_supports_universal_builds -> return_true_if_universal_builds_are_supported_on_this_system
_supports_arm64_builds -> return_true_if_arm64_builds_are_supported_on_this_system
_remove_universal_flags -> remove_all_universal_build_arguments_from_config_vars
_override_all_archs -> allow_override_of_all_archs_with_ARCHFLAGS_env_var
S_ISDIR -> return_true_if_mode_is_from_a_directory
S_ISCHR -> return_true_if_mode_is_from_a_character_special_device_file
S_ISBLK -> return_true_if_mode_is_from_a_block_special_device_file
S_ISREG -> return_true_if_mode_is_from_a_regular_file
S_ISFIFO -> return_true_if_mode_is_from_a_FIFO_named_pipe
S_ISLNK -> return_true_if_mode_is_from_a_symbolic_link
S_ISSOCK -> return_true_if_mode_is_from_a_socket
Int2AP -> convert_integer_to_A_P_string_representation
_create_temporary -> create_a_temp_file_based_on_path_and_open_for_reading_and_writing
_sync_close -> close_file_ensuring_all_changes_are_physically_on_disk
localize -> parse_string_as_locale_number_according_to_locale_settings
atof -> parse_string_as_float_according_to_locale_settings
atoi -> convert_string_to_integer_according_to_locale_settings
getlocale -> getlocale_and_return_current_setting_for_given_locale_category_as_tuple
resetlocale -> resetlocale_and_set_locale_for_category_to_default_value
testsource -> extract_test_sources_from_doctest_docstring_as_script
debug_src -> debug_src_and_debug_single_doctest_docstring_in_argument_src
win_getpass -> win_getpass_and_prompt_for_password_with_echo_off
whichmodule -> whichmodule_and_find_module_an_object_belongs_to
openpty -> openpty_and_open_a_pty_master_slave_pair_using_os_openpty_if_possible
master_open -> master_open_and_open_a_pty_master_return_fd_and_filename_of_slave_end
_copy -> _copy_and_parent_copy_loop_copies_pty_master_standard_output_standard_input
spawn -> spawn_and_run_another_program_in_a_new_process_specified_as_command_list_cmd
contextmanager -> contextmanager_decorator_typical_usage
some_generator -> some_generator_with_setup_yield_value_and_finally_cleanup
helper -> helper_and_return_generator_context_manager_instance
asynccontextmanager -> asynccontextmanager_decorator_typical_usage
some_async_generator -> some_async_generator_with_setup_yield_value_and_finally_cleanup
_exact_ratio -> _exact_ratio_and_return_real_number_x_to_exact_numerator_denominator_pair
_find_lteq -> _find_lteq_and_locate_leftmost_value_exactly_equal_to_x
_find_rteq -> _find_rteq_and_locate_rightmost_value_exactly_equal_to_x
mean -> mean_and_return_sample_arithmetic_mean_of_data
count -> count_and_return_the_number_of_elements_in_data
geometric_mean -> geometric_mean_and_return_geometric_mean_of_data
stdev -> stdev_and_return_square_root_of_sample_variance
pstdev -> pstdev_and_return_square_root_of_population_variance
asdict -> asdict_and_return_fields_of_dataclass_instance_as_dictionary
astuple -> astuple_and_return_fields_of_dataclass_instance_as_tuple
_remove_dups_flatten -> _remove_dups_flatten_and_internal_helper_for_Union_creation_and_substitution
no_type_check_decorator -> no_type_check_decorator_and_decorator_to_give_another_decorator_the_no_type_check_effect
wrapped_decorator -> wrapped_decorator_and_wrap_the_decorator_with_something_that_wraps_the_decorated_function_in_no_type_check
DER_cert_to_PEM_cert -> DER_cert_to_PEM_cert_and_take_certificate_in_binary_DER_format_and_return_PEM_version_as_string
_ymd2ord -> _ymd2ord_and_convert_year_month_day_to_ordinal_considering_01_Jan_0001_as_day_1
localcontext -> extended_context_manager_returns_context_manager
sin -> calculate_sine_with_extended_context
_normalize -> adjust_operands_and_return_normalized
_decimal_lshift_exact -> compute_integer_after_left_shift
_log10_lb -> lower_bound_for_log_base_10_of_positive_integer
_group_lengths -> convert_localeconv_grouping_to_iterable
_ifconfig_getnode -> get_hardware_address_using_ifconfig
_ip_getnode -> get_hardware_address_using_ip
_arp_getnode -> get_hardware_address_using_arp
_lanscan_getnode -> get_hardware_address_using_lanscan
_netstat_getnode -> get_hardware_address_using_netstat
_ipconfig_getnode -> deprecated_get_hardware_address_on_windows
_netbios_getnode -> deprecated_get_hardware_address_on_windows
init_builtin -> deprecated_load_and_return_builtin_module
readmailcapfile -> deprecated_read_mailcapfile_use_getcaps
parseline -> parse_one_entry_in_mailcap_file_return_dictionary
_run_module_code -> helper_to_run_code_in_new_namespace
insertion_unsort -> insertion_unsort_coding_with_oldchar
generate_integers -> bias_adaptation_generate_integers
insertion_sort -> insertion_unsort_coding_with_char
normalize_encoding -> normalize_encoding_name_ascii_only
_find_all_simple -> find_all_files_under_path
get_versions -> try_to_find_versions_of_gcc_ld_dllwrap
copy_xxmodule_c -> helper_for_tests_copying_xxmodule_c
test_compile -> test_compile_copying_xxmodule_c
find_tzfile -> retrieve_path_to_tzif_file_from_key
get_libpaths -> on_aix_buildtime_searchpath_stored_in_executable
find_shared -> search_directories_for_archive_and_return_result
_inject -> inject_paths_with_suffix
framework_find -> find_framework_using_dyld_semantics_loose_manner
doModuleCleanups -> execute_all_module_cleanup_functions_normally_called_for_you_after_tear_down_module
skipIf -> skip_test_if_condition_is_true
skipUnless -> skip_test_unless_condition_is_true
log_to_stderr -> turn_on_logging_and_add_stderr_handler
_cleanup_tests -> cleanup_multiprocessing_resources_when_tests_completed
SocketClient -> return_connection_object_connected_to_socket_given_by_address
import_main_path -> set_sys_modules___main___to_module_at_main_path
RawArray -> return_ctypes_array_allocated_from_shared_memory
public_methods -> return_list_of_names_of_methods_of_obj_which_do_not_start_with_underscore
AutoProxy -> return_an_auto_proxy_for_token
request_host -> return_request_host_as
build_opener -> create_an_opener_object_from_list_of_handlers
proxy_bypass_environment -> test_if_proxies_should_not_be_used_for_particular_host
urldefrag -> remove_any_existing_fragment_from_url
to_text_string -> convert_obj_to_unicode_text_string
to_binary_string -> convert_obj_to_binary_string
qbytearray_to_str -> convert_qbytearray_object_to_str_compatible_with_python_2_3
get_module_path -> return_module_modname_base_path
get_changeset -> return_mercurial_repository_path_revision_number
prepend_modules_to_path -> prepend_to_sys_path_all_modules_located_in_module_base_path
qapplication -> return_qapplication_instance
keybinding -> return_keybinding
onerror -> error_handler_for_shutil_rmtree
generate_requirements -> convert_requirements_from_setup_style_dictionary
dedent_description -> dedent_and_convert_pkg_info_description_to_unicode
_get_cache_dir -> return_persistent_or_temporary_cache_directory
backup_dir -> figure_out_name_of_directory_to_back_up_given_dir_to
egg_link_path_from_sys_path -> look_for_egg_link_file_for_project_name_by_walking_sys_path
test_writable_dir -> check_if_directory_is_writable
get_best_invocation_for_this_pip -> try_to_figure_out_best_way_to_invoke_pip_in_current_environment
guess_lexer -> determine_lexer_by_text_with_options_and_handle_exceptions
get_all_formatters -> retrieve_all_formatter_classes_and_handle_cache
get_formatter_by_name -> lookup_and_instantiate_formatter_by_alias_with_exception
get_all_styles -> generate_generator_for_all_styles_both_builtin_and_plugin
compat_system -> attempt_to_get_build_system_backend_and_requirements
default_subprocess_runner -> execute_subprocess_with_default_settings
_supported_features -> list_supported_options_features_supported_by_backend
get_requires_for_build_wheel -> invoke_optional_hook_get_requires_for_build_wheel
get_requires_for_build_editable -> invoke_optional_hook_get_requires_for_build_editable
prepare_metadata_for_build_wheel -> invoke_optional_hook_prepare_metadata_for_build_wheel
prepare_metadata_for_build_editable -> invoke_optional_hook_prepare_metadata_for_build_editable
build_wheel -> invoke_mandatory_hook_build_wheel_with_copy_if_prebuilt
build_editable -> invoke_optional_hook_build_editable_with_copy_if_prebuilt
get_requires_for_build_sdist -> invoke_optional_hook_get_requires_for_build_sdist
build_sdist -> invoke_mandatory_hook_build_sdist
get_dependent_dists -> generate_list_of_dependent_distributions_recursive
get_required_dists -> generate_list_of_required_distributions_recursive
interpret -> evaluate_marker_and_return_result_with_context_handling
finder_for_path -> return_resource_finder_for_path_with_cache_handling
intranges_from_list -> represent_list_of_integers_as_sequence_of_ranges
intranges_contain -> determine_if_integer_falls_into_one_of_ranges
merge_cookies -> add_cookies_to_cookiejar_and_return_merged_cookiejar
merge_hooks -> properly_merge_request_and_session_hooks_with_handling
options -> send_options_request_with_optional_arguments_handling
head -> send_head_request_with_optional_arguments_handling
post -> send_post_request_with_data_and_json_handling
put -> send_put_request_with_data_handling
patch -> send_patch_request_with_data_handling
delete -> send_delete_request_with_optional_arguments_handling
parse_list_header -> parse_lists_as_described_by_rfc_with_quoted_string_handling
virtualenv_no_global -> check_if_running_under_venv_and_return_no_global_flag
warn_if_run_as_root -> issue_warning_for_sudo_users_on_unix
_get_prepared_distribution -> prepare_distribution_for_install_requirement_with_tracker
install_editable -> run_setup_py_develop_for_package_in_editable_mode
get_legacy_build_wheel_path -> return_path_to_legacy_wheel_in_temp_dir
generate_editable_metadata -> generate_metadata_for_build_editable_using_pep660
build_wheel_editable -> build_wheel_for_install_requirement_using_pep660
build_wheel_pep517 -> build_wheel_for_install_requirement_using_pep517
deduce_helpful_msg -> return_helpful_message_for_requirements_file
find_path_to_project_root_from_repo_root -> find_project_root_relative_to_repo_root
_infer_user -> try_to_find_user_scheme_for_current_platform
_parse_links_html5lib -> parse_html_document_and_yield_anchor_elements
transform_hits -> convert_list_of_versions_into_list_of_packages
_import_module -> import_module_and_return_last_module
lookup -> look_for_encoding_by_label
iter_decode -> pull_based_decoder_for_unicode_strings
iter_encode -> pull_based_encoder_for_unicode_strings
html_doctype_matches -> check_if_file_has_html_doctype
guess_decode_from_terminal -> decode_text_based_on_terminal_encoding
bygroups -> callback_that_yields_actions_for_each_group_in_match
callback -> callback_that_processes_matched_groups
regex_opt -> compiled_regex_matching_strings_in_list
apply_filters -> apply_filters_to_stream_using_given_filters
get_filter_by_name -> return_instantiated_filter_by_name
get_all_filters -> generate_all_filter_names
get_all_lexers -> generate_all_lexer_info_tuples
find_lexer_class -> lookup_lexer_class_by_name
find_lexer_class_by_name -> lookup_lexer_class_by_alias
get_lexer_for_filename -> get_lexer_for_filename_and_code
get_lexer_for_mimetype -> get_lexer_for_mimetype_and_options
add_dict_to_cookiejar -> add_dictionary_to_cookie_jar_with_given_cookies
get_unicode_from_response -> retrieve_unicode_content_from_response_object_and_handle_warnings
address_in_network -> check_if_ip_belongs_to_network_subnet_and_return_boolean
select_proxy -> choose_proxy_for_given_url_from_provided_proxy_dictionary
resolve_proxies -> resolve_proxies_based_on_request_and_configuration_input
default_headers -> generate_default_headers_as_case_insensitive_dictionary
prepend_scheme_if_needed -> prepend_given_scheme_to_url_if_not_present
urldefragauth -> remove_fragment_and_authentication_from_url
pick_unit_and_suffix -> choose_unit_and_suffix_for_given_size_and_suffix_list
reconfigure -> replace_global_console_with_another_console_instance
set_cell_size -> adjust_string_length_to_fit_within_given_number_of_cells
format_header_param_html5 -> format_and_quote_single_header_param_using_html5_strategy
connection_from_url -> create_connection_pool_instance_for_given_url
create_proxy_ssl_context -> generate_ssl_context_for_given_ssl_version_and_cert_requirements
wait_for_write -> wait_for_writing_to_be_available_on_given_socket
_const_compare_digest_backport -> compare_two_digests_of_equal_length_in_constant_time
_encode_target -> percent_encode_request_target_to_avoid_invalid_characters
is_connection_dropped -> check_if_connection_dropped_and_should_be_closed
_ipaddress_match -> perform_exact_matching_of_ip_addresses
is_appengine_sandbox -> report_if_app_running_in_first_generation_sandbox
inject_into_urllib3 -> monkey_patch_urllib3_with_pyopenssl_ssl_support
_create_cfstring_array -> create_cfmutablearray_from_list_of_python_binary_data
_assert_no_error -> check_return_code_and_throw_exception_on_error
load_cdll -> load_dynamic_link_library_by_name_with_fallback_path
safe_version -> convert_arbitrary_string_to_standard_version_string
parse_requirements -> yield_requirement_objects_for_each_specification_in_strings
split_sections -> split_string_or_iterable_into_section_content_pairs
distros_for_url -> yield_egg_or_source_distribution_objects_for_given_url
unpack_zipfile -> unpack_zip_file_to_given_extract_directory
unpack_tarfile -> unpack_tar_or_targz_or_tarbz2_file_to_given_extract_directory
find_parent_package -> find_parent_package_with_sorted_and_common_ancestors
strip_marker -> strip_marker_and_avoid_calling_pip_with_environment_marker
hide_setuptools -> remove_references_to_setuptools_modules_and_allow_appropriate_import
_have_cython -> check_if_cython_can_be_imported_and_return_true_or_false
get_unpatched_class -> protect_against_repatching_distutils_and_ensure_no_prior_patch
_msvc14_find_vc2015 -> find_vc2015_for_msvc14_and_return_best_version_and_directory
msvc14_get_vc_env -> patched_get_vc_env_for_msvc14_and_support_extra_compilers
msvc14_gen_lib_options -> patched_gen_lib_options_for_msvc14_and_fix_compatibility
load_group -> given_value_return_entry_point_as_entry_point
check_nsp -> verify_valid_namespace_packages_and_warn_if_not_declared
check_package_data -> verify_value_as_dict_of_package_names_to_glob_lists
unpack -> move_everything_under_source_directory_to_destination_directory
_pathlib_compat -> convert_path_like_objects_to_filename_for_compatibility
_special_method_cache -> install_wrapper_method_and_return_simple_proxy
proxy -> create_proxy_to_installed_wrapper_method_and_use_it
retry_call -> trap_specified_exceptions_for_up_to_specified_retries
install -> add_backport_distribution_finder_to_sys_meta_path
matches -> check_if_finder_module_is_frozen_importlib_external_and_has_find_distributions
consumer -> advance_reverse_iterator_to_first_yield_point_automatically
tally -> print_thing_number_and_state_using_reverse_iterator
repeat_last -> return_sequence_of_elements_and_then_return_none
filter_except -> yield_items_that_validator_does_not_raise_specified_exceptions_for
map_except -> transform_items_with_function_and_yield_result_except_specified_exceptions
nth_product -> compute_product_at_sort_position_index_without_computing_previous_products
pad_none -> return_sequence_of_elements_and_then_return_none
ncycles -> return_sequence_elements_n_times
repeatfunc -> call_function_repeatedly_and_return_iterable_over_results
first_true -> return_first_true_value_in_iterable
normalize_path -> normalize_path_by_ensuring_it_is_only_a_file_name
_some_attrgetter -> extended_attribute_getter_for_truthy_values
_acessor -> attribute_accessor_with_items
canonic_data_files -> canonicalize_data_files_for_setup_py
walk_egg -> egg_directory_contents_walker_without_metadata
is_python_script -> determine_if_text_is_entire_python_script
only_strs -> filter_only_string_values
run_commands -> execute_all_commands_in_distribution
scheme -> override_selected_posix_prefix_scheme
_load_schemes -> extend_and_load_schemes
_inject_headers -> resolve_and_inject_headers_for_scheme
repeat_each -> repeat_each_element_in_iterable
zip_equal -> zip_iterables_and_raise_error_on_unequal_lengths
chunked_even -> break_iterable_into_approximately_equal_length_lists
triplewise -> return_overlapping_triplets_from_iterable
do_override -> ensure_local_distutils_override
showinfo -> display_info_message_with_title
showwarning -> display_warning_message_with_title
askquestion -> prompt_user_with_question_and_title
askokcancel -> ask_if_operation_should_proceed_with_ok_cancel
askyesno -> ask_user_yes_or_no_question_with_title
askyesnocancel -> ask_user_yes_no_cancel_question_with_title
askretrycancel -> ask_if_operation_should_be_retried_with_retry
mainloop -> run_tcl_main_loop_with_iteration_limit
getboolean -> convert_tcl_object_to_boolean
_val_or_dict -> format_options_and_call_tk_command
tclobjs_to_py -> convert_tcl_objects_to_python
unescape -> unescape_string_with_entities_dictionary
parseString -> parse_string_into_dom_from_string
parse_headers -> parse_rfc2822_headers_from_file_pointer
iso2time -> parse_iso8601_time_from_text
iso2time -> parse_iso8601_time_from_text_and_return
user_domain_match -> check_blocking_accepting_domains_for_match
request_path -> get_path_component_of_request_uri_as_string
reach -> return_reach_of_host_as_string
bootstrap -> bootstrap_pip_into_python_installation
resolve_name -> resolve_relative_module_name_to_absolute
_requires_builtin -> verify_module_is_builtin_decorator
_requires_builtin_wrapper -> wrapper_to_verify_module_is_builtin
_requires_frozen -> verify_module_is_frozen_decorator
_requires_frozen_wrapper -> wrapper_to_verify_module_is_frozen
_module_repr -> implementation_of_module_type_repr
spec_from_loader -> create_module_spec_based_on_loader_methods
module_from_spec -> create_module_based_on_spec
_load -> return_new_module_loaded_by_loader
_gcd_import -> import_and_return_module_based_on_name
_path_isfile -> replacement_for_os_path_isfile
_path_isdir -> replacement_for_os_path_isdir
_write_atomic -> write_data_to_path_atomically
_calc_mode -> calculate_mode_permissions_for_bytecode
_validate_timestamp_pyc -> validate_timestamp_based_pyc_against_source
_code_to_timestamp_pyc -> produce_data_for_timestamp_based_pyc
_code_to_hash_pyc -> produce_data_for_hash_based_pyc
_install -> install_path_based_import_components
loads -> convert_xml_rpc_packet_to_unmarshalled_data
_get_headnode_dict -> return_dictionary_of_head_node_types
get_fixers_from_package -> return_fully_qualified_names_for_fixers
Call -> create_function_call_node
ImportAndCall -> return_import_statement_and_method_call_node
is_tuple -> check_if_node_represents_tuple_literal
in_special_context -> check_if_node_is_in_iterable_context
does_tree_import -> does_tree_import_checking_top_level
fixup_parse_tree -> fixup_parse_tree_ensure_suite_format
fixup_simple_stmt -> fixup_simple_stmt_semicolon_handling
load_packaged_grammar -> load_pickled_grammar_from_package_or_file
tokenize -> tokenize_with_readline_and_tokeneater
get_end_linenumber -> get_end_linenumber_of_text_widget
idle_showwarning -> show_warning_in_idle_format
idle_showwarning_subproc -> show_warning_in_idle_format_subprocess
show_socket_error -> show_socket_error_with_messagebox
get_line_info -> get_line_info_tuple_indent_codeline
build_bits -> build_bits_for_platform
reformat_comment -> reformat_comment_with_header_limit
isCarbonTk -> is_carbon_tk_used
isXQuartz -> is_xquartz_used
tkVersionWarning -> tk_version_warning_message
preferTabsPreferenceWarning -> prefer_tabs_preference_warning_message
setupApp -> setup_app_perform_customizations
wrap_info -> wrap_info_replace_traceback_id
close_remote_debugger -> close_remote_debugger_shutdown_link
format_selection -> format_selection_apply_format_line
apply -> apply_format_line_to_selected_lines
create_task -> create_task_schedule_coroutine
run_coroutine_threadsafe -> run_coroutine_threadsafe_submit_to_loop
get_running_loop -> get_running_loop_raise_runtime_error
set_event_loop -> set_event_loop_via_event_loop_policy
new_event_loop -> new_event_loop_via_event_loop_policy
get_child_watcher -> get_child_watcher_via_event_loop_policy
set_child_watcher -> set_child_watcher_via_event_loop_policy
_set_concurrent_future_state -> set_concurrent_future_state_from_future
_copy_future_state -> copy_future_state_from_another_future
does_tree_import -> does_tree_import_checking_top_level
fixup_parse_tree -> fixup_parse_tree_ensure_suite_format
fixup_simple_stmt -> fixup_simple_stmt_semicolon_handling
load_packaged_grammar -> load_pickled_grammar_from_package_or_file
tokenize -> tokenize_with_readline_and_tokeneater
get_end_linenumber -> get_end_linenumber_of_text_widget
idle_showwarning -> show_warning_in_idle_format
idle_showwarning_subproc -> show_warning_in_idle_format_subprocess
show_socket_error -> show_socket_error_with_messagebox
get_line_info -> get_line_info_tuple_indent_codeline
build_bits -> build_bits_for_platform
reformat_comment -> reformat_comment_with_header_limit
isCarbonTk -> is_carbon_tk_used
isXQuartz -> is_xquartz_used
tkVersionWarning -> tk_version_warning_message
preferTabsPreferenceWarning -> prefer_tabs_preference_warning_message
setupApp -> setup_app_perform_customizations
wrap_info -> wrap_info_replace_traceback_id
close_remote_debugger -> close_remote_debugger_shutdown_link
format_selection -> format_selection_apply_format_line
apply -> apply_format_line_to_selected_lines
create_task -> create_task_schedule_coroutine
run_coroutine_threadsafe -> run_coroutine_threadsafe_submit_to_loop
get_running_loop -> get_running_loop_raise_runtime_error
set_event_loop -> set_event_loop_via_event_loop_policy
new_event_loop -> new_event_loop_via_event_loop_policy
get_child_watcher -> get_child_watcher_via_event_loop_policy
set_child_watcher -> set_child_watcher_via_event_loop_policy
_set_concurrent_future_state -> set_concurrent_future_state_from_future
_copy_future_state -> copy_future_state_from_another_future
parsedate -> convert_time_string_to_time_tuple
header_encode -> encode_single_header_with_base64_in_given_charset
translate -> translate_symbolic_cnf_to_numbered_cnf_and_return_reverse_mapping
from_dnf -> convert_from_or_of_ands_to_and_of_ors_in_cnf
parse_grid -> convert_grid_to_dict_of_possible_values_or_return_false_if_contradiction_detected
configuration_to_dict -> return_dict_of_configuration_data_gathered_by_given_handlers
wrap_future -> wrap_given_concurrent_futures_Future_object
_future_repr_info -> helper_function_for_Future_repr_with_exception_information_and_callbacks
fileConfig -> read_logging_configuration_from_ConfigParser_format_file
addLevelName -> associate_levelName_with_level_for_message_formatting
_removeHandlerRef -> remove_handler_reference_from_internal_cleanup_list
_addHandlerRef -> add_handler_to_internal_cleanup_list_using_weak_reference
fatal -> don_t_use_this_function_use_critical_instead
exception -> log_message_with_severity_ERROR_on_root_logger_with_exception_information
warning -> log_message_with_severity_WARNING_on_root_logger
info -> log_message_with_severity_INFO_on_root_logger
debug -> log_message_with_severity_DEBUG_on_root_logger
log -> log_msg_with_integer_severity_level_on_root_logger
get_qcontent -> qcontent_equals_qtext_or_quoted_pair
get_atom -> atom_equals_optional_CFWS_one_or_more_atext_optional_CFWS
get_dot_atom -> dot_atom_equals_optional_CFWS_dot_atom_text_optional_CFWS
get_addr_spec -> addr_spec_equals_local_part_at_domain
get_name_addr -> name_addr_equals_optional_display_name_angle_addr
get_invalid_mailbox -> read_everything_up_to_one_of_the_chars_in_endchars_outside_formal_grammar
get_no_fold_literal -> no_fold_literal_equals_bracket_zero_or_more_dtext_bracket
parse_message_id -> message_id_equals_Message_ID_msg_id_CRLF
get_invalid_parameter -> read_everything_up_to_next_semicolon_outside_formal_grammar
get_token -> token_equals_optional_CFWS_one_ttext_optional_CFWS
get_attribute -> optional_CFWS_one_or_more_attrtext_optional_CFWS
get_value -> quoted_string_or_attribute
_find_mime_parameters -> do_our_best_to_find_parameters_in_invalid_MIME_header
encode_base64 -> encode_message_payload_in_Base64_and_add_Content_Transfer_Encoding_header
encode_quopri -> encode_message_payload_in_quoted_printable_and_add_Content_Transfer_Encoding_header
getaddresses -> return_list_of_REALNAME_EMAIL_for_each_fieldvalue
format_datetime -> turn_datetime_into_date_string_as_specified_in_RFC_2822'''
names_dict = {}
for [x, y] in list(map(lambda x: x.split(' -> '), lst.split('\n'))):
  names_dict[x] = y


func_data['bad_name'] = func_data['func_name'].apply(lambda x: names_dict[x] if x in names_dict.keys() else "")
func_data = func_data.drop_duplicates(subset = 'func_name')
# func_data.to_csv('functions_data.csv')

In [13]:
func_data =func_data.drop_duplicates(subset = 'func_name')
list(func_data[func_data['bad_name']== ""]['func_definition'])[:30]

[]

In [54]:
dataset[dataset['code'].str.count('pager')>2]

,index,code,prompt,bad_prompt,bad_code,prompt_names_dict,numerical_prompt,numerical_code,prompt_numerical_dict
83,83,"def pager(text):\n """"""The first time this i...","def pager(text):\n """"""The first time this i...",def determine_and_use_appropriate_pager(text):...,def determine_and_use_appropriate_determine_an...,"{""pager"": ""determine_and_use_appropriate_pager...","def 0(text):\n """"""The first time this is ca...","def 0(text):\n """"""The first time this is ca...","{""pager"": ""0"", ""getpager"": ""1""}"


## Make code and promt data

In [16]:
dataset['bad_prompt'] = ""
dataset['bad_code'] = ""
dataset['prompt_names_dict'] = ""
for i in range(dataset.shape[0]):
  line = dataset.loc[i]
  p = line['prompt']
  c = line['code']
  d = {}
  for func_name, func in extract_function_code_with_regex(line['prompt']).items():
    
    if func_name in line['code']:
      d[func_name] = names_dict[func_name]
      p = p.replace(func_name+"(", names_dict[func_name]+"(" )
      c = c.replace(func_name+"(", names_dict[func_name]+"(" )
  for func_name, func in extract_function_code_with_regex(line['code']).items():
      c = c.replace(func_name+"(", names_dict[func_name]+"(")
  dataset.at[i, 'bad_prompt'] = p
  dataset.at[i, 'bad_code'] = c
  dataset.at[i,'prompt_names_dict'] = json.dumps(d)


In [23]:
dataset['numerical_prompt'] = ""
dataset['numerical_code'] = ""
dataset['prompt_numerical_dict'] = ""
for i in range(dataset.shape[0]):
  line = dataset.loc[i]
  p = line['prompt']
  c = line['code']
  d = {}
  for num, func_name in enumerate(extract_function_code_with_regex(line['prompt'])):
    if func_name in line['code']:
      d[func_name] = str(num)
      p = p.replace(func_name+"(", str(num)+"(")
      c = c.replace(func_name+"(", str(num)+"(")
        

  for n, func_name in enumerate(extract_function_code_with_regex(line['code'])):
      c = c.replace(func_name+"(", str(1+n+num)+"(")
  dataset.at[i, 'numerical_prompt'] = p
  dataset.at[i, 'numerical_code'] = c
  dataset.at[i,'prompt_numerical_dict'] = json.dumps(d)

In [24]:
dataset.head()

,index,code,prompt,bad_prompt,bad_code,prompt_names_dict,numerical_prompt,numerical_code,prompt_numerical_dict
0,0,"def cellname(x, y):\n ""Translate a cell coo...","def str(val):\n """"""Convert float to string,...","def locale_aware_float_to_string(val):\n """"...","def generate_cell_name(x, y):\n ""Translate ...","{""str"": ""locale_aware_float_to_string"", ""colnu...","def 0(val):\n """"""Convert float to string, t...","def 2(x, y):\n ""Translate a cell coordinate...","{""str"": ""0"", ""colnum2name"": ""1""}"
1,1,"def test_basic():\n ""Basic non-gui self-tes...","def cellname(x, y):\n ""Translate a cell coo...","def generate_cell_name(x, y):\n ""Translate ...","def perform_basic_non_gui_self_test():\n ""B...","{""cellname"": ""generate_cell_name""}","def 0(x, y):\n ""Translate a cell coordinate...","def 1():\n ""Basic non-gui self-test.""\n ...","{""cellname"": ""0""}"
2,2,"def display_menu(stdscr, menu_y):\n ""Displa...","def erase_menu(stdscr, menu_y):\n ""Clear th...","def erase_menu_clears_menu_space(stdscr, menu_...",def display_menu_commands_and_instructions(std...,"{""erase_menu"": ""erase_menu_clears_menu_space""}","def 0(stdscr, menu_y):\n ""Clear the space w...","def 1(stdscr, menu_y):\n ""Display the menu ...","{""erase_menu"": ""0""}"
3,3,"def merge_profile():\n """"""Reads sys.getdxp(...","def has_pairs(profile):\n """"""Returns True i...","def check_for_pairs(profile):\n """"""Returns ...",def merge_profile_into_module_cached_copy():\n...,"{""has_pairs"": ""check_for_pairs""}","def 0(profile):\n """"""Returns True if the Py...","def 1():\n """"""Reads sys.getdxp() and merges...","{""has_pairs"": ""0""}"
4,4,"def snapshot_profile():\n """"""Returns the cu...","def merge_profile():\n """"""Reads sys.getdxp(...",def merge_profile_into_module_cached_copy():\n...,"def get_cumulative_execution_profile():\n ""...","{""merge_profile"": ""merge_profile_into_module_c...","def 0():\n """"""Reads sys.getdxp() and merges...","def 1():\n """"""Returns the cumulative execut...","{""merge_profile"": ""0""}"


In [36]:
dataset.loc[18:21]

,index,code,prompt,bad_prompt,bad_code,prompt_names_dict,numerical_prompt,numerical_code,prompt_numerical_dict
18,18,"def iter_importers(fullname=""""):\n """"""Yield...","def get_importer(path_item):\n """"""Retrieve ...",def retrieve_finder_for_path_item(path_item):\...,"def yield_finders_for_module(fullname=""""):\n ...","{""get_importer"": ""retrieve_finder_for_path_item""}","def 0(path_item):\n """"""Retrieve a finder fo...","def 1(fullname=""""):\n """"""Yield finders for ...","{""get_importer"": ""0""}"
19,19,"def _split_optional_netmask(address):\n """"""...","def str(val):\n """"""Convert float to string,...","def locale_aware_float_to_string(val):\n """"...",def split_netmask_and_raise_error_if_needed(ad...,"{""str"": ""locale_aware_float_to_string""}","def 0(val):\n """"""Convert float to string, t...","def 1(address):\n """"""Helper to split the ne...","{""str"": ""0""}"
20,20,"def _find_lines(code, strs):\n """"""Return li...","def _find_lines(code, strs):\n """"""Return li...","def get_line_numbers_for_code_objects(code, st...","def get_line_numbers_for_code_objects(code, st...","{""_find_lines"": ""get_line_numbers_for_code_obj...","def 0(code, strs):\n """"""Return lineno dict ...","def 0(code, strs):\n """"""Return lineno dict ...","{""_find_lines"": ""0"", ""_find_lines_from_code"": ..."
21,21,"def distb(tb=None, *, file=None):\n """"""Disa...","def disassemble(co, lasti=-1, *, file=None):\n...","def disassemble_code_object(co, lasti=-1, *, f...","def disassemble_traceback(tb=None, *, file=Non...","{""disassemble"": ""disassemble_code_object""}","def 0(co, lasti=-1, *, file=None):\n """"""Dis...","def 1(tb=None, *, file=None):\n """"""Disassem...","{""disassemble"": ""0""}"


In [25]:
dataset.to_csv('code_data.csv')

In [55]:
code_string = dataset.loc[83]['code']
dataset.loc[83]['prompt']

'def pager(text):\n    """The first time this is called, determine what kind of pager to use."""\n    global pager\n    pager = getpager()\n    pager(text)\n\ndef getpager():\n    """Decide what method to use for paging through text."""\n    if not hasattr(sys.stdin, "isatty"):\n        return plainpager\n    if not hasattr(sys.stdout, "isatty"):\n        return plainpager\n    if not sys.stdin.isatty() or not sys.stdout.isatty():\n        return plainpager\n    use_pager = os.environ.get(\'MANPAGER\') or os.environ.get(\'PAGER\')\n    if use_pager:\n        if sys.platform == \'win32\': # pipes completely broken in Windows\n            return lambda text: tempfilepager(plain(text), use_pager)\n        elif os.environ.get(\'TERM\') in (\'dumb\', \'emacs\'):\n            return lambda text: pipepager(plain(text), use_pager)\n        else:\n            return lambda text: pipepager(text, use_pager)\n    if os.environ.get(\'TERM\') in (\'dumb\', \'emacs\'):\n        return plainpager\n   

In [56]:
pattern = r'pager\('
lines = code_string.split('\n')
matching_lines = [i for i, line in enumerate(lines) if re.search(pattern, line)]
print(matching_lines)

[0, 3, 4]


In [61]:
d = pd.read_csv('clean_naming/logs/generation_data_1701997206.5867755.csv')
d[d['name_type']=='Original']['answer'].sum()

125

In [62]:
d[d['name_type']=='GPT generated']['answer'].sum()

79

In [63]:
d[d['name_type']=='Numerical']['answer'].sum()

160

In [64]:
[1,2,3][:0]

[]